In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

# 画像の読み込み、train/test分離



In [2]:
from skimage import data, color, feature
import skimage.data
%run path.py

In [3]:
ch = data.chelsea()

type(ch)

numpy.ndarray

In [4]:
img_array = skimage.io.imread(img_path + "OK/IMG_6753.JPG")

# 白黒化、縮小

In [5]:
from skimage.transform import rescale
image_rescaled = rescale(img_array, 1.0 / 4.0, multichannel=True, anti_aliasing=True)

c:\program files\python37\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [6]:
image_rescaled.shape

(756, 1008, 3)

# HOG計算

In [7]:
hog = feature.hog(image_rescaled, block_norm='L1')

In [8]:
hog.shape

(924048,)

In [9]:
from itertools import chain



In [10]:
import os 

OK_name_list = os.listdir(img_path + "/OK")



In [11]:
#メモリが足りなくなる!
#OK_image_data = np.vstack([ rescale(skimage.io.imread(img_path + "OK/" + name ), 1.0 / 4.0, mode='constant', multichannel=True, anti_aliasing=True) \
#                           for name in OK_name_list])